In [2]:
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
#Sklearn imports(and othrs)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.cluster import KMeans
from sklearn.metrics import median_absolute_error

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.datasets import make_classification 
from datetime import datetime

In [2]:
filepath_wa = "./data/Metropolitan-street.parquet"

# read the Parquet file into a pandas dataframe
df_ward = pq.read_pandas(filepath_wa).to_pandas()
barnet_df = df_ward[df_ward['LSOA name'].str.contains('Barnet')==True]
barnetBur_df = barnet_df[barnet_df['Crime type'].str.contains('Burglary')==True]
barnetBur_df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
2508,48d7fae1e76d275cfecabf17efbf6fecf769966089dcae...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.205163,51.655067,On or near Salisbury Road,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN
2509,d1d37e3e233d650abaf3eae1d6585e33f706cebe857ffc...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.203868,51.653824,On or near Union Street,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN
2532,9b09776a838d0d532429a57b320021da8bcffac6a2aa53...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.210051,51.657237,On or near Wentworth Road,E01000249,Barnet 001B,Burglary,Investigation complete; no suspect identified,NaN
2666,5c63da75bb8aa5fe30c925d6a4e58c3608bdd6fe68f33a...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.181376,51.648343,On or near Station Road,E01000251,Barnet 002D,Burglary,Investigation complete; no suspect identified,NaN
2667,6906ae484ce7bec7133bfdfa918cdcda566e04d71c465f...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.175660,51.651015,On or near Lytton Road,E01000251,Barnet 002D,Burglary,Investigation complete; no suspect identified,NaN


In [3]:
df = pd.read_csv('./data/LSOA_WARD.csv')
df=df[["LSOA21NM","WD22NM"]]
df= df[df['LSOA21NM'].str.contains('Barnet')==True]
e=set(df['LSOA21NM'])
f = set(barnetBur_df['LSOA name'])



In [4]:

merged_df = pd.merge(left=barnetBur_df, right=df, left_on='LSOA name', right_on='LSOA21NM')
merged_df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,LSOA21NM,WD22NM
0,48d7fae1e76d275cfecabf17efbf6fecf769966089dcae...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.205163,51.655067,On or near Salisbury Road,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN,Barnet 001A,High Barnet
1,d1d37e3e233d650abaf3eae1d6585e33f706cebe857ffc...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.203868,51.653824,On or near Union Street,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN,Barnet 001A,High Barnet
2,6b13c905e40eccf73feec600e87a6c70e0667b10980e95...,2020-07,Metropolitan Police Service,Metropolitan Police Service,-0.207839,51.654289,On or near The Avenue,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN,Barnet 001A,High Barnet
3,9869e99b7803ce398498332b862347626619671258ecdb...,2021-01,Metropolitan Police Service,Metropolitan Police Service,-0.205204,51.656578,On or near Strafford Road,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN,Barnet 001A,High Barnet
4,83c73a8dfb5749e2f3922ff87ae87e3f0422e3cd8c88fa...,2021-01,Metropolitan Police Service,Metropolitan Police Service,-0.203575,51.655735,On or near Shopping Area,E01000248,Barnet 001A,Burglary,Investigation complete; no suspect identified,NaN,Barnet 001A,High Barnet


In [5]:
df_deprivation = pd.read_csv("./data/Deprivation.csv")
df_deprivation = pd.merge(left=df_deprivation, right=df, left_on='LSOA name (2011)', right_on='LSOA21NM')
df_deprivation.columns
df_deprivation=df_deprivation.drop(['LSOA code (2011)', 'LSOA name (2011)','Local Authority District code (2019)','LSOA21NM'],axis=1)
df_deprivation=df_deprivation.groupby(['WD22NM']).median()
df_deprivation.head()


,Index of Multiple Deprivation (IMD) Score,Income Score (rate),Employment Score (rate),"Education, Skills and Training Score",Health Deprivation and Disability Score,Crime Score,Barriers to Housing and Services Score,Living Environment Score,Income Deprivation Affecting Children Index (IDACI) Score (rate),Income Deprivation Affecting Older People (IDAOPI) Score (rate),Children and Young People Sub-domain Score,Adult Skills Sub-domain Score,Geographical Barriers Sub-domain Score,Wider Barriers Sub-domain Score,Indoors Sub-domain Score,Outdoors Sub-domain Score,Dependent Children aged 0-15: mid 2015 (excluding prisoners),Population aged 16-59: mid 2015 (excluding prisoners),Older population aged 60 and over: mid 2015 (excluding prisoners),Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners)
WD22NM,,,,,,,,,,,,,,,,,,,,
Barnet Vale,12.365,0.0865,0.0655,2.7115,-1.218,0.105,28.4555,21.657,0.0910,0.1100,-1.1335,0.191,0.0485,2.4530,-0.012,0.610,306.0,943.0,354.0,932.5
Brunswick Park,9.982,0.0745,0.0545,5.2565,-1.329,-0.318,22.9290,17.954,0.0695,0.1255,-0.7670,0.232,-0.4545,2.1115,-0.302,0.552,329.5,952.5,369.5,956.5
Burnt Oak,27.055,0.1850,0.1060,21.3480,-0.464,0.481,38.6570,27.146,0.2030,0.3100,-0.0990,0.361,-0.6750,5.0300,0.033,1.039,412.0,1111.0,254.0,1098.0
Childs Hill,16.669,0.1020,0.0770,4.7980,-1.087,0.348,30.8650,29.849,0.0980,0.1520,-0.5730,0.175,-0.4820,4.0200,0.226,1.143,314.0,1025.0,299.0,1012.0
Colindale North,33.625,0.2650,0.1520,22.2120,-0.077,0.085,44.2690,19.735,0.3030,0.3830,0.2030,0.340,-0.2340,5.3470,-0.321,0.738,546.0,1081.0,215.0,1035.0


In [6]:
df_final = pd.merge(left=merged_df, right=df_deprivation, left_on='WD22NM', right_on='WD22NM')
df_final.head()


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,...,Children and Young People Sub-domain Score,Adult Skills Sub-domain Score,Geographical Barriers Sub-domain Score,Wider Barriers Sub-domain Score,Indoors Sub-domain Score,Outdoors Sub-domain Score,Dependent Children aged 0-15: mid 2015 (excluding prisoners),Population aged 16-59: mid 2015 (excluding prisoners),Older population aged 60 and over: mid 2015 (excluding prisoners),Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners)
0,48d7fae1e76d275cfecabf17efbf6fecf769966089dcae...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.205163,51.655067,On or near Salisbury Road,E01000248,Barnet 001A,Burglary,...,-0.952,0.192,0.056,1.896,-0.206,0.439,313.0,855.0,457.0,864.0
1,d1d37e3e233d650abaf3eae1d6585e33f706cebe857ffc...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.203868,51.653824,On or near Union Street,E01000248,Barnet 001A,Burglary,...,-0.952,0.192,0.056,1.896,-0.206,0.439,313.0,855.0,457.0,864.0
2,6b13c905e40eccf73feec600e87a6c70e0667b10980e95...,2020-07,Metropolitan Police Service,Metropolitan Police Service,-0.207839,51.654289,On or near The Avenue,E01000248,Barnet 001A,Burglary,...,-0.952,0.192,0.056,1.896,-0.206,0.439,313.0,855.0,457.0,864.0
3,9869e99b7803ce398498332b862347626619671258ecdb...,2021-01,Metropolitan Police Service,Metropolitan Police Service,-0.205204,51.656578,On or near Strafford Road,E01000248,Barnet 001A,Burglary,...,-0.952,0.192,0.056,1.896,-0.206,0.439,313.0,855.0,457.0,864.0
4,83c73a8dfb5749e2f3922ff87ae87e3f0422e3cd8c88fa...,2021-01,Metropolitan Police Service,Metropolitan Police Service,-0.203575,51.655735,On or near Shopping Area,E01000248,Barnet 001A,Burglary,...,-0.952,0.192,0.056,1.896,-0.206,0.439,313.0,855.0,457.0,864.0


In [7]:
df_crimes = df_final[["Month", "Crime ID","WD22NM"]].copy()
df_crimes= df_crimes.groupby(["WD22NM","Month"]).count()


def rolling_sum2(group,y,str,s):
    group[str] = group['Crime ID'].rolling(window=y).sum().shift(s)
    return group

df_crimes = df_crimes.groupby('WD22NM', as_index=False).apply(lambda x: rolling_sum2(x,1,"1Months",1))
df_crimes.fillna(0, inplace=True)
df_crimes = df_crimes.groupby('WD22NM', as_index=False).apply(lambda x: rolling_sum2(x,1,"12MonthsLag",12))
df_crimes.fillna(0, inplace=True)
df_crimes = df_crimes.groupby('WD22NM', as_index=False).apply(lambda x: rolling_sum2(x,4,"4MonthsLag",4))
df_crimes.fillna(0, inplace=True)
df_crimes = df_crimes.groupby('WD22NM', as_index=False).apply(lambda x: rolling_sum2(x,4,"4MonthsLagYearAgo",16))
df_crimes.fillna(0, inplace=True)



#Merge the count of crimes to the main dataframe
merged_df = pd.merge(df_final, df_crimes, on=['Month', 'WD22NM'], how='left')
merged_df.head()



C:\Users\20211445\AppData\Local\Temp\ipykernel_18104\3860000195.py:9: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_crimes = df_crimes.groupby('WD22NM', as_index=False).apply(lambda x: rolling_sum2(x,1,"1Months",1))
C:\Users\20211445\AppData\Local\Temp\ipykernel_18104\3860000195.py:11: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

,Crime ID_x,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,...,Outdoors Sub-domain Score,Dependent Children aged 0-15: mid 2015 (excluding prisoners),Population aged 16-59: mid 2015 (excluding prisoners),Older population aged 60 and over: mid 2015 (excluding prisoners),Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners),Crime ID_y,1Months,12MonthsLag,4MonthsLag,4MonthsLagYearAgo
0,48d7fae1e76d275cfecabf17efbf6fecf769966089dcae...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.205163,51.655067,On or near Salisbury Road,E01000248,Barnet 001A,Burglary,...,0.439,313.0,855.0,457.0,864.0,6,2.0,11.0,44.0,29.0
1,d1d37e3e233d650abaf3eae1d6585e33f706cebe857ffc...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.203868,51.653824,On or near Union Street,E01000248,Barnet 001A,Burglary,...,0.439,313.0,855.0,457.0,864.0,6,2.0,11.0,44.0,29.0
2,6b13c905e40eccf73feec600e87a6c70e0667b10980e95...,2020-07,Metropolitan Police Service,Metropolitan Police Service,-0.207839,51.654289,On or near The Avenue,E01000248,Barnet 001A,Burglary,...,0.439,313.0,855.0,457.0,864.0,5,3.0,8.0,34.0,31.0
3,9869e99b7803ce398498332b862347626619671258ecdb...,2021-01,Metropolitan Police Service,Metropolitan Police Service,-0.205204,51.656578,On or near Strafford Road,E01000248,Barnet 001A,Burglary,...,0.439,313.0,855.0,457.0,864.0,7,5.0,13.0,33.0,36.0
4,83c73a8dfb5749e2f3922ff87ae87e3f0422e3cd8c88fa...,2021-01,Metropolitan Police Service,Metropolitan Police Service,-0.203575,51.655735,On or near Shopping Area,E01000248,Barnet 001A,Burglary,...,0.439,313.0,855.0,457.0,864.0,7,5.0,13.0,33.0,36.0


In [8]:
### File with no dummies
#merged_df.to_parquet('final_test.parquet')

In [9]:
encoded_wards = pd.get_dummies(df_final['WD22NM'])
df_months = pd.get_dummies(df_final['Month'].str.split("-").str[-1].astype(int))
df_months.columns = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'okt', 'nov', 'dec']
encoded_wards.columns
df_mon = df_months.join(encoded_wards)
df_mon

,jan,feb,mar,apr,may,jun,jul,aug,sep,okt,...,Golders Green,Hendon,High Barnet,Mill Hill,Totteridge & Woodside,Underhill,West Finchley,West Hendon,Whetstone,Woodhouse
0,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42506,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42507,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42508,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42509,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
merged_df["Month"]=pd.to_datetime(merged_df["Month"])
merged_df["Year"]=merged_df["Month"].dt.year


merged_df=merged_df.join(df_mon)


In [11]:
merged_df.columns

Index(['Crime ID_x', 'Month', 'Reported by', 'Falls within', 'Longitude',
       'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type',
       'Last outcome category', 'Context', 'LSOA21NM', 'WD22NM',
       'Index of Multiple Deprivation (IMD) Score', 'Income Score (rate)',
       'Employment Score (rate)', 'Education, Skills and Training Score',
       'Health Deprivation and Disability Score', 'Crime Score',
       'Barriers to Housing and Services Score', 'Living Environment Score',
       'Income Deprivation Affecting Children Index (IDACI) Score (rate)',
       'Income Deprivation Affecting Older People (IDAOPI) Score (rate)',
       'Children and Young People Sub-domain Score',
       'Adult Skills Sub-domain Score',
       'Geographical Barriers Sub-domain Score',
       'Wider Barriers Sub-domain Score', 'Indoors Sub-domain Score',
       'Outdoors Sub-domain Score',
       'Dependent Children aged 0-15: mid 2015 (excluding prisoners)',
       'Population aged 16-59: 

In [12]:
df_lm=merged_df.drop(['jan','WD22NM'], axis=1)
y = merged_df['Crime ID_y']
X=df_lm.drop(['Crime ID_y'],axis=1)
# X=df_lm[['Living Environment Score','Geographical Barriers Sub-domain Score','Indoors Sub-domain Score','Outdoors Sub-domain Score','Population aged 16-59: mid 2015 (excluding prisoners)','Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners) ','1Months','12MonthsLag','feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'okt', 'nov', 'dec','Year',
#         'Barnet Vale', 'Brunswick Park', 'Burnt Oak', 'Childs Hill',
#        'Colindale North', 'Cricklewood', 'East Barnet', 'East Finchley',
#        'Edgware', 'Edgwarebury', 'Finchley Church End', 'Friern Barnet',
#        'Garden Suburb', 'Golders Green', 'Hendon', 'High Barnet', 'Mill Hill',
#        'Totteridge & Woodside', 'Underhill', 'West Finchley', 'West Hendon',
#        'Whetstone','4MonthsLag','4MonthsLagYearAgo']]
#X=df.drop(['Crime ID_y','Mid-2020 population','Area Sq Km'],axis=1)

X=df_lm[['Living Environment Score','Geographical Barriers Sub-domain Score','Indoors Sub-domain Score','Outdoors Sub-domain Score','Population aged 16-59: mid 2015 (excluding prisoners)','Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners) ','1Months','12MonthsLag','feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'okt', 'nov', 'dec','Year',
        'Barnet Vale', 'Brunswick Park', 'Burnt Oak', 'Childs Hill',
       'Colindale North', 'Cricklewood', 'East Barnet', 'East Finchley',
       'Edgware', 'Edgwarebury', 'Finchley Church End', 'Friern Barnet',
       'Garden Suburb', 'Golders Green', 'Hendon', 'High Barnet', 'Mill Hill',
       'Totteridge & Woodside', 'Underhill', 'West Finchley', 'West Hendon',
       'Whetstone','4MonthsLag','4MonthsLagYearAgo']]


In [13]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,
                                               random_state=42, shuffle=True)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# calculate R-squared for the predictions
r2 = r2_score(y_test, y_pred)
print("R-squared: ", r2)
mse = mean_squared_error(y_test, y_pred)
print("Mean squared error: ", mse)
mdae = median_absolute_error(y_test, y_pred)
print("Median absolute error: ", mdae)


R-squared:  0.5424767464781941
Mean squared error:  32.391739888827445
Median absolute error:  3.598583221435547


In [14]:
model.coef_


array([-4.97979021e+05, -1.22349903e+09, -2.50334828e+09,  1.23769671e+10,
       -1.07763002e+09,  1.09530928e+09,  4.68856374e-01, -3.24715161e-02,
       -1.16817690e+00, -1.25294119e-01, -3.85328147e+00, -2.97518463e+00,
       -2.98438243e+00, -3.49839359e+00, -2.42274457e+00, -2.71833384e+00,
       -2.21490523e-01,  1.38315493e+00,  1.83874693e-01, -9.06902300e-02,
        1.38038987e+10, -2.87140960e+09,  7.49252131e+09,  8.44635993e+09,
        4.75432593e+10,  7.44866421e+08, -7.70694698e+09,  4.73604001e+09,
       -3.60663014e+09,  5.29916883e+09, -8.44303699e+09,  5.24253294e+09,
       -8.42084797e+09, -2.31063807e+09, -6.82453744e+09, -4.35984402e+09,
        4.75361798e+09, -8.88451329e+09,  7.56182245e+09,  3.00093629e+09,
       -1.51704132e+10, -1.34773634e+10,  9.20270830e-02, -1.75544620e-03])

In [15]:

#"2Months"
#"4Months"
# compute the correlation matrix
corr_matrix = df_lm.corr()['Crime ID_y']
# Set display options for pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Print the correlation matrix
print(corr_matrix)

C:\Users\20211445\AppData\Local\Temp\ipykernel_18104\1035634366.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df_lm.corr()['Crime ID_y']


Longitude                                                                                             0.009475
Latitude                                                                                             -0.180605
Context                                                                                                    NaN
Index of Multiple Deprivation (IMD) Score                                                             0.012257
Income Score (rate)                                                                                  -0.025057
Employment Score (rate)                                                                              -0.028655
Education, Skills and Training Score                                                                 -0.039926
Health Deprivation and Disability Score                                                              -0.036876
Crime Score                                                                                           0.067548
B

In [16]:
df_lm.columns

Index(['Crime ID_x', 'Month', 'Reported by', 'Falls within', 'Longitude', 'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type', 'Last outcome category', 'Context', 'LSOA21NM', 'Index of Multiple Deprivation (IMD) Score', 'Income Score (rate)', 'Employment Score (rate)', 'Education, Skills and Training Score', 'Health Deprivation and Disability Score', 'Crime Score', 'Barriers to Housing and Services Score', 'Living Environment Score', 'Income Deprivation Affecting Children Index (IDACI) Score (rate)', 'Income Deprivation Affecting Older People (IDAOPI) Score (rate)', 'Children and Young People Sub-domain Score', 'Adult Skills Sub-domain Score', 'Geographical Barriers Sub-domain Score', 'Wider Barriers Sub-domain Score', 'Indoors Sub-domain Score', 'Outdoors Sub-domain Score', 'Dependent Children aged 0-15: mid 2015 (excluding prisoners)', 'Population aged 16-59: mid 2015 (excluding prisoners)', 'Older population aged 60 and over: mid 2015 (excluding prisoners)',
       'Worki

In [17]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming you have your feature matrix X and target variable y

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and fit the Lasso model
lasso = Lasso(alpha=0.1)  # Adjust the alpha parameter for regularization strength
lasso.fit(X_train_scaled, y_train)

# Evaluate the model
train_score = lasso.score(X_train_scaled, y_train)
test_score = lasso.score(X_test_scaled, y_test)

# Print the coefficients
coefficients = lasso.coef_
intercept = lasso.intercept_
print("Coefficients:", coefficients)
print("Intercept:", intercept)
print("Train Score:", train_score)
print("Test Score:", test_score)


Coefficients: [ 0.10933498 -0.04554444  0.16900822  0.04964368  0.          0.08278183
  3.88704079 -0.         -0.03606253  0.09228649 -0.69418437 -0.44477872
 -0.4319471  -0.5604807  -0.29686524 -0.37489696  0.07988667  0.5312694
  0.16715132 -0.26186837  0.          0.          0.          0.1731896
 -0.09647626 -0.08473802 -0.         -0.         -0.         -0.0369394
  0.01311984  0.          0.         -0.          0.0401941  -0.
 -0.         -0.         -0.03983034  0.05090741 -0.         -0.
  2.28037542 -0.        ]
Intercept: 14.00026464361326
Train Score: 0.5252371514189725
Test Score: 0.5380065435884676


In [18]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming you have your feature matrix X and target variable y

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and fit the Ridge model
ridge = Ridge(alpha=1.0)  # Adjust the alpha parameter for regularization strength
ridge.fit(X_train_scaled, y_train)

# Evaluate the model
train_score = ridge.score(X_train_scaled, y_train)
test_score = ridge.score(X_test_scaled, y_test)

# Print the coefficients
coefficients = ridge.coef_
intercept = ridge.intercept_
print("Coefficients:", coefficients)
print("Intercept:", intercept)
print("Train Score:", train_score)
print("Test Score:", test_score)


Coefficients: [ 0.03089009 -0.02505483  0.14739821  0.26218799 -0.04401319 -0.04751509
  3.77634671 -0.27498746 -0.33449884 -0.03641629 -1.01099642 -0.78114725
 -0.77309852 -0.89599272 -0.63388772 -0.69417667 -0.06221015  0.41292898
  0.05397004 -0.31763322  0.13368508  0.11788442  0.16220539  0.37707127
 -0.13823917 -0.19636717  0.01495251 -0.02376588  0.0546367  -0.11386835
  0.21708997  0.19842527  0.03398156  0.07817846  0.24647082 -0.0119056
  0.08026633 -0.02769061 -0.05347793  0.26332601  0.11101873  0.02660604
  2.57857213 -0.05298699]
Intercept: 14.000264643613258
Train Score: 0.5301351692904382
Test Score: 0.5424770187683421


In [4]:
filepath_wa = "./data/final_test.parquet"

# read the Parquet file into a pandas dataframe
df_ward = pq.read_pandas(filepath_wa).to_pandas()
df_ward

,Crime ID_x,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,...,Wider Barriers Sub-domain Score,Indoors Sub-domain Score,Outdoors Sub-domain Score,Dependent Children aged 0-15: mid 2015 (excluding prisoners),Population aged 16-59: mid 2015 (excluding prisoners),Older population aged 60 and over: mid 2015 (excluding prisoners),Working age population 18-59/64: for use with Employment Deprivation Domain (excluding prisoners),Crime ID_y,1Months,12MonthsLag
0,48d7fae1e76d275cfecabf17efbf6fecf769966089dcae...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.205163,51.655067,On or near Salisbury Road,E01000248,Barnet 001A,Burglary,...,1.896,-0.2060,0.4390,313.0,855.0,457.0,864.0,6,2.0,11.0
1,d1d37e3e233d650abaf3eae1d6585e33f706cebe857ffc...,2020-05,Metropolitan Police Service,Metropolitan Police Service,-0.203868,51.653824,On or near Union Street,E01000248,Barnet 001A,Burglary,...,1.896,-0.2060,0.4390,313.0,855.0,457.0,864.0,6,2.0,11.0
2,6b13c905e40eccf73feec600e87a6c70e0667b10980e95...,2020-07,Metropolitan Police Service,Metropolitan Police Service,-0.207839,51.654289,On or near The Avenue,E01000248,Barnet 001A,Burglary,...,1.896,-0.2060,0.4390,313.0,855.0,457.0,864.0,5,3.0,8.0
3,9869e99b7803ce398498332b862347626619671258ecdb...,2021-01,Metropolitan Police Service,Metropolitan Police Service,-0.205204,51.656578,On or near Strafford Road,E01000248,Barnet 001A,Burglary,...,1.896,-0.2060,0.4390,313.0,855.0,457.0,864.0,7,5.0,13.0
4,83c73a8dfb5749e2f3922ff87ae87e3f0422e3cd8c88fa...,2021-01,Metropolitan Police Service,Metropolitan Police Service,-0.203575,51.655735,On or near Shopping Area,E01000248,Barnet 001A,Burglary,...,1.896,-0.2060,0.4390,313.0,855.0,457.0,864.0,7,5.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42506,3838bdfda0c7dfeb2074ceaa4d1fef6197e7bd81c7ee75...,2020-02,Metropolitan Police Service,Metropolitan Police Service,-0.221855,51.570569,On or near Claremont Way,E01000223,Barnet 039C,Burglary,...,4.487,-0.2405,1.5395,357.5,984.5,201.0,974.0,14,11.0,27.0
42507,c8c49db70e8382ef5d5031e1599378ef9c9dcd1cd0202d...,2020-02,Metropolitan Police Service,Metropolitan Police Service,-0.221855,51.570569,On or near Claremont Way,E01000223,Barnet 039C,Burglary,...,4.487,-0.2405,1.5395,357.5,984.5,201.0,974.0,14,11.0,27.0
42508,a2cafd97e586a7cf93809eb3139937926bc3212ed0ba7e...,2020-03,Metropolitan Police Service,Metropolitan Police Service,-0.221388,51.567738,On or near Brent Terrace,E01000223,Barnet 039C,Burglary,...,4.487,-0.2405,1.5395,357.5,984.5,201.0,974.0,5,14.0,8.0
42509,9b2024e56df3a0b0352b5dccd2303d0833a62508de3671...,2020-03,Metropolitan Police Service,Metropolitan Police Service,-0.229716,51.570283,On or near Shopping Area,E01000223,Barnet 039C,Burglary,...,4.487,-0.2405,1.5395,357.5,984.5,201.0,974.0,5,14.0,8.0
